In [1]:
import sqlite3
import csv
from tabulate import tabulate

Establish connection

In [2]:
# Make a connection to SQLite database (or create it)
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()

# Create table (adjust columns to match your CSV)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales (
        Date, Day, Month, Year, Customer_Age, Age_Group,	
        Customer_Gender, Country, State, Product_Category,
        Sub_Category, Product, Order_Quantity, Unit_Cost,
        Unit_Price, Profit, Cost, Revenue
    )
''')

Load the csv file

In [3]:
# Load data from CSV file
with open('sales_data.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        cursor.execute('INSERT INTO sales VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', row)
conn.commit()

In [17]:
# display first 5 rows
cursor.execute('SELECT * FROM sales LIMIT 5')
rows = cursor.fetchall()
column_names = [description[0] for description in cursor.description]
print(tabulate(rows, headers=column_names))

Date        Day    Month     Year    Customer_Age    Age_Group       Customer_Gender    Country    State             Product_Category    Sub_Category    Product              Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
----------  -----  --------  ------  --------------  --------------  -----------------  ---------  ----------------  ------------------  --------------  -------------------  ----------------  -----------  ------------  --------  ------  ---------
Date        Day    Month     Year    Customer_Age    Age_Group       Customer_Gender    Country    State             Product_Category    Sub_Category    Product              Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
11/26/2013  26     November  2013    19              Youth (<25)     M                  Canada     British Columbia  Accessories         Bike Racks      Hitch Rack - 4-Bike  8                 45           120           590       360     950
11/26/2015  26     Nov

In [117]:
# Check all columns for any missing values
cursor.execute('PRAGMA table_info(sales)')
columns = cursor.fetchall()
for column in columns:
    col_name = column[1]
    cursor.execute(f'SELECT COUNT(*) FROM sales WHERE {col_name} IS NULL OR {col_name} = ""')
    missing_count = cursor.fetchone()[0]
    print(f"Column '{col_name}' has {missing_count} missing values.")

Column 'Date' has 0 missing values.
Column 'Day' has 0 missing values.
Column 'Month' has 0 missing values.
Column 'Year' has 0 missing values.
Column 'Customer_Age' has 0 missing values.
Column 'Age_Group' has 0 missing values.
Column 'Customer_Gender' has 0 missing values.
Column 'Country' has 0 missing values.
Column 'State' has 0 missing values.
Column 'Product_Category' has 0 missing values.
Column 'Sub_Category' has 0 missing values.
Column 'Product' has 0 missing values.
Column 'Order_Quantity' has 0 missing values.
Column 'Unit_Cost' has 0 missing values.
Column 'Unit_Price' has 0 missing values.
Column 'Profit' has 0 missing values.
Column 'Cost' has 0 missing values.
Column 'Revenue' has 0 missing values.


In [119]:
# Map F to Female and M to Male in Customer_Gender column
cursor.execute('UPDATE sales SET Customer_Gender = "Female" WHERE Customer_Gender = "F"')
cursor.execute('UPDATE sales SET Customer_Gender = "Male" WHERE Customer_Gender = "M"')
conn.commit()

# Verify the changes
cursor.execute('SELECT DISTINCT Customer_Gender FROM sales')
rows = cursor.fetchall()
print(tabulate(rows, headers=['Customer_Gender']))

Customer_Gender
-----------------
Customer_Gender
Male
Female


Sorting by Single Column

In [18]:
# Sort by Revenue in descending order and display top 5 rows
cursor.execute('SELECT * FROM sales ORDER BY Revenue DESC LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date        Day    Month     Year    Customer_Age    Age_Group             Customer_Gender    Country    State     Product_Category    Sub_Category     Product                  Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
----------  -----  --------  ------  --------------  --------------------  -----------------  ---------  --------  ------------------  ---------------  -----------------------  ----------------  -----------  ------------  --------  ------  ---------
Date        Day    Month     Year    Customer_Age    Age_Group             Customer_Gender    Country    State     Product_Category    Sub_Category     Product                  Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
2/28/2016   28     February  2016    34              Young Adults (25-34)  M                  Australia  Victoria  Accessories         Hydration Packs  Hydration Pack - 70 oz.  23                21           55            516       483     999
4/3/2016  

In [19]:
# Sort by Revenue in ascending order and display top 5 rows
cursor.execute('SELECT * FROM sales ORDER BY Revenue ASC LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date         Day  Month      Year    Customer_Age  Age_Group             Customer_Gender    Country    State             Product_Category    Sub_Category       Product                  Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
---------  -----  -------  ------  --------------  --------------------  -----------------  ---------  ----------------  ------------------  -----------------  ---------------------  ----------------  -----------  ------------  --------  ------  ---------
10/2/2013      2  October    2013              29  Young Adults (25-34)  F                  Canada     British Columbia  Accessories         Bottles and Cages  Water Bottle - 30 oz.                 2            2             5         6       4         10
6/28/2016     28  June       2016              44  Adults (35-64)        M                  Canada     British Columbia  Accessories         Bottles and Cages  Water Bottle - 30 oz.                 2            2             5      

In [20]:
# Sort by Revenue in ascending order and display top 5 rows
cursor.execute('SELECT * FROM sales ORDER BY Revenue ASC LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date         Day  Month      Year    Customer_Age  Age_Group             Customer_Gender    Country    State             Product_Category    Sub_Category       Product                  Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
---------  -----  -------  ------  --------------  --------------------  -----------------  ---------  ----------------  ------------------  -----------------  ---------------------  ----------------  -----------  ------------  --------  ------  ---------
10/2/2013      2  October    2013              29  Young Adults (25-34)  F                  Canada     British Columbia  Accessories         Bottles and Cages  Water Bottle - 30 oz.                 2            2             5         6       4         10
6/28/2016     28  June       2016              44  Adults (35-64)        M                  Canada     British Columbia  Accessories         Bottles and Cages  Water Bottle - 30 oz.                 2            2             5      

Sorting by Multiple Columns

In [21]:
# Sort by Profit descending Revenue in ascending order
# Sort by Revenue in ascending order and display top 5 rows
cursor.execute('SELECT * FROM sales ORDER BY Profit DESC, Revenue ASC LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date       Day    Month     Year    Customer_Age    Age_Group       Customer_Gender    Country    State    Product_Category    Sub_Category    Product                Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
---------  -----  --------  ------  --------------  --------------  -----------------  ---------  -------  ------------------  --------------  ---------------------  ----------------  -----------  ------------  --------  ------  ---------
Date       Day    Month     Year    Customer_Age    Age_Group       Customer_Gender    Country    State    Product_Category    Sub_Category    Product                Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
8/10/2012  10     August    2012    41              Adults (35-64)  F                  Germany    Hessen   Bikes               Road Bikes      Road-350-W Yellow, 48  2                 1083         1701          998       2166    3164
6/6/2016   6      June      2016    41              A

In [22]:
# Sort by Revenue in descending order and age_group in ascending display top 5 rows
cursor.execute('SELECT * FROM sales ORDER BY Revenue DESC, Age_group ASC LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date        Day    Month     Year    Customer_Age    Age_Group             Customer_Gender    Country    State     Product_Category    Sub_Category     Product                  Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
----------  -----  --------  ------  --------------  --------------------  -----------------  ---------  --------  ------------------  ---------------  -----------------------  ----------------  -----------  ------------  --------  ------  ---------
Date        Day    Month     Year    Customer_Age    Age_Group             Customer_Gender    Country    State     Product_Category    Sub_Category     Product                  Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
12/26/2013  26     December  2013    37              Adults (35-64)        F                  Australia  Victoria  Accessories         Hydration Packs  Hydration Pack - 70 oz.  23                21           55            516       483     999
12/14/2015

Using Limit with OFFSET

In [23]:
# Sort by Revenue in descending order and age_group in ascending display top 5 rows
cursor.execute('SELECT * FROM sales ORDER BY Revenue DESC, Age_group ASC LIMIT 5 OFFSET 10')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date         Day  Month      Year    Customer_Age  Age_Group             Customer_Gender    Country    State           Product_Category    Sub_Category    Product                           Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
---------  -----  -------  ------  --------------  --------------------  -----------------  ---------  --------------  ------------------  --------------  ------------------------------  ----------------  -----------  ------------  --------  ------  ---------
4/26/2011     26  April      2011              29  Young Adults (25-34)  M                  France     Hauts de Seine  Bikes               Road Bikes      Road-150 Red, 62                               3         2171          3578      3470    6513       9983
5/27/2016     27  May        2016              29  Young Adults (25-34)  M                  Germany    Hamburg         Bikes               Road Bikes      Road-150 Red, 44                               3         2171    

Filtering data Using WHERE

In [24]:
cursor.execute('SELECT * FROM sales WHERE Country = "United States" LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date         Day  Month      Year    Customer_Age  Age_Group             Customer_Gender    Country        State       Product_Category    Sub_Category    Product                Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
---------  -----  -------  ------  --------------  --------------------  -----------------  -------------  ----------  ------------------  --------------  -------------------  ----------------  -----------  ------------  --------  ------  ---------
3/27/2014     27  March      2014              51  Adults (35-64)        M                  United States  Oregon      Accessories         Bike Racks      Hitch Rack - 4-Bike                 9           45           120       524     405        929
3/27/2016     27  March      2016              51  Adults (35-64)        M                  United States  Oregon      Accessories         Bike Racks      Hitch Rack - 4-Bike                 7           45           120       407     315        722
7/19

Using WHERE and Conditional Statement

In [25]:
# Using AND to filter data
cursor.execute('SELECT * FROM sales WHERE Country = "United States" AND State = "California" LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date         Day  Month      Year    Customer_Age  Age_Group       Customer_Gender    Country        State       Product_Category    Sub_Category    Product                Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
---------  -----  -------  ------  --------------  --------------  -----------------  -------------  ----------  ------------------  --------------  -------------------  ----------------  -----------  ------------  --------  ------  ---------
7/13/2013     13  July       2013              43  Adults (35-64)  F                  United States  California  Accessories         Bike Racks      Hitch Rack - 4-Bike                10           45           120       726     450       1176
7/13/2015     13  July       2015              43  Adults (35-64)  F                  United States  California  Accessories         Bike Racks      Hitch Rack - 4-Bike                10           45           120       726     450       1176
3/31/2014     31  March     

In [26]:
# Use OR to filter data
cursor.execute('SELECT * FROM sales WHERE Country = "United States" OR Country = "Canada" LIMIT 5 OFFSET 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date         Day  Month        Year    Customer_Age  Age_Group             Customer_Gender    Country    State             Product_Category    Sub_Category    Product                Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
---------  -----  ---------  ------  --------------  --------------------  -----------------  ---------  ----------------  ------------------  --------------  -------------------  ----------------  -----------  ------------  --------  ------  ---------
9/2/2015       2  September    2015              29  Young Adults (25-34)  M                  Canada     British Columbia  Accessories         Bike Racks      Hitch Rack - 4-Bike                 1           45           120        74      45        119
1/22/2014     22  January      2014              29  Young Adults (25-34)  M                  Canada     British Columbia  Accessories         Bike Racks      Hitch Rack - 4-Bike                 1           45           120        74      45

Returning some columns using Alias

In [27]:
# Returning some columns using Alias
cursor.execute('SELECT Date, Country, State, Product_Category AS Category, Revenue AS Sales FROM sales LIMIT 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date        Day        Month             Year              Customer_Age
----------  ---------  ----------------  ----------------  --------------
Date        Country    State             Product_Category  Revenue
11/26/2013  Canada     British Columbia  Accessories       950
11/26/2015  Canada     British Columbia  Accessories       950
3/23/2014   Australia  New South Wales   Accessories       2401
3/23/2016   Australia  New South Wales   Accessories       2088


Data Manipulation with SQL (INSERT, UPDATE, DELETE)

In [33]:
# Adding a new sale record
cursor.execute('''INSERT INTO sales (Date, Day, Month, Year, Customer_Age, Age_Group,    
        Customer_Gender, Country, State, Product_Category,
        Sub_Category, Product, Order_Quantity, Unit_Cost,
        Unit_Price, Profit, Cost, Revenue) VALUES
        ('1/1/2020', 'Wednesday', 'January', '2020', '25', 'Young Adults',
        'Female', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '1', '500', '1000', '500', '500', '1000')''')
conn.commit()

In [39]:
# Adding multiple records
cursor.executemany('''INSERT INTO sales (Date, Day, Month, Year, Customer_Age, Age_Group,    
        Customer_Gender, Country, State, Product_Category,
        Sub_Category, Product, Order_Quantity, Unit_Cost,
        Unit_Price, Profit, Cost, Revenue) VALUES
        (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', [
    ('2/1/2020', 'Thursday', 'February', '2020', '30', 'Adults',
        'Male', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '8', '650', '900', '500', '500', '1000'),
    ('3/1/2020', 'Friday', 'March', '2020', '35', 'Adults',
        'Male', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '5', '550', '1100', '500', '500', '1000'),
    ('4/1/2020', 'Saturday', 'April', '2020', '40', 'Adults',
        'Female', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '4', '400', '800', '500', '500', '1000'),
    ('5/1/2020', 'Sunday', 'May', '2020', '45', 'Adults',
        'Male', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '2', '380', '750', '500', '500', '1000'),
    ('6/1/2020', 'Monday', 'June', '2020', '50', 'Adults',
        'Male', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '3', '700', '1200', '500', '500', '1000')
])
conn.commit()

# Verify the new record
cursor.execute('SELECT * FROM sales WHERE Product = "Sofas" LIMIT 5 OFFSET 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date      Day       Month       Year    Customer_Age  Age_Group    Customer_Gender    Country        State       Product_Category    Sub_Category    Product      Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
--------  --------  --------  ------  --------------  -----------  -----------------  -------------  ----------  ------------------  --------------  ---------  ----------------  -----------  ------------  --------  ------  ---------
6/1/2020  Monday    June        2020              50  Adults       Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
2/1/2020  Thursday  February    2020              30  Adults       Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
3/1/2020  Friday    March       2020              35  Adults       F

In [38]:
# Add another values to the table
cursor.executemany('INSERT INTO sales VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', [
    ('7/1/2020', 'Tuesday', 'July', '2020', '55', 'Adults',
        'Female', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '2', '500', '1000', '500', '500', '1000'),
    ('8/1/2020', 'Wednesday', 'August', '2020', '60', 'Adults',
        'Male', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '1', '500', '1000', '500', '500', '1000')
])
conn.commit()

# Verify the new records
cursor.execute('SELECT * FROM sales WHERE Product = "Sofas" LIMIT 10 OFFSET 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date      Day        Month       Year    Customer_Age  Age_Group     Customer_Gender    Country        State       Product_Category    Sub_Category    Product      Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
--------  ---------  --------  ------  --------------  ------------  -----------------  -------------  ----------  ------------------  --------------  ---------  ----------------  -----------  ------------  --------  ------  ---------
6/1/2020  Monday     June        2020              50  Adults        Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
2/1/2020  Thursday   February    2020              30  Adults        Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
3/1/2020  Friday     March       2020              35  Adult

Update

In [40]:
# Update a record
cursor.execute('''UPDATE sales SET Revenue = 1500, Profit = 700 WHERE Date = '1/1/2020' AND Product = 'Sofas' ''')
conn.commit()

# Verify the updated record
cursor.execute('SELECT * FROM sales WHERE Product = "Sofas" LIMIT 10 OFFSET 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date      Day        Month       Year    Customer_Age  Age_Group     Customer_Gender    Country        State       Product_Category    Sub_Category    Product      Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
--------  ---------  --------  ------  --------------  ------------  -----------------  -------------  ----------  ------------------  --------------  ---------  ----------------  -----------  ------------  --------  ------  ---------
6/1/2020  Monday     June        2020              50  Adults        Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
2/1/2020  Thursday   February    2020              30  Adults        Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
3/1/2020  Friday     March       2020              35  Adult

DELETE

In [41]:
# Delete a record
cursor.execute('''DELETE FROM sales WHERE Date = '8/1/2020' AND Product = 'Sofas' ''')
conn.commit()

# Verify the deleted record
cursor.execute('SELECT * FROM sales WHERE Product = "Sofas" LIMIT 10 OFFSET 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date      Day        Month       Year    Customer_Age  Age_Group     Customer_Gender    Country        State       Product_Category    Sub_Category    Product      Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
--------  ---------  --------  ------  --------------  ------------  -----------------  -------------  ----------  ------------------  --------------  ---------  ----------------  -----------  ------------  --------  ------  ---------
6/1/2020  Monday     June        2020              50  Adults        Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
2/1/2020  Thursday   February    2020              30  Adults        Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       500     500       1000
3/1/2020  Friday     March       2020              35  Adult

In [43]:
# Delete a multiple records
cursor.execute('''DELETE FROM sales WHERE Date IN ('2/1/2020', '3/1/2020', '4/1/2020') AND Product = 'Sofas' ''')
conn.commit()

# Verify the deleted records
cursor.execute('SELECT * FROM sales WHERE Product = "Sofas" LIMIT 10 OFFSET 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date      Day        Month      Year    Customer_Age  Age_Group     Customer_Gender    Country        State       Product_Category    Sub_Category    Product      Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
--------  ---------  -------  ------  --------------  ------------  -----------------  -------------  ----------  ------------------  --------------  ---------  ----------------  -----------  ------------  --------  ------  ---------
1/1/2020  Wednesday  January    2020              25  Young Adults  Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       700     500       1500
1/1/2020  Wednesday  January    2020              25  Young Adults  Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       700     500       1500
5/1/2020  Sunday     May        2020              45  Adults    

To delete the whole from the table, run the code below in the python enviroment

# Delete whole records
cursor.execute('DELETE FROM sales')
conn.commit()

# Verify all records are deleted
cursor.execute('SELECT * FROM sales')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Use Transactions for Atomicity

In [54]:
# Begin a transaction
conn.execute('BEGIN TRANSACTION;')
cursor.execute('SELECT * FROM sales WHERE Product = "Sofas" LIMIT 10 OFFSET 5')
# Update a record
cursor.execute('''UPDATE sales SET Revenue = 1500, Profit = 700 WHERE Date = '1/1/2020' AND Product = 'Sofas' ''')
# Delete a record
cursor.execute('''DELETE FROM sales WHERE Date = '8/1/2020' AND Product = 'Sofas' ''')
# Insert a new record
cursor.execute('''INSERT INTO sales (Date, Day, Month, Year, Customer_Age, Age_Group,    
        Customer_Gender, Country, State, Product_Category,
        Sub_Category, Product, Order_Quantity, Unit_Cost,
        Unit_Price, Profit, Cost, Revenue) VALUES
        ('7/1/2020', 'Tuesday', 'July', '2020', '55', 'Adults',
        'Female', 'United States', 'California', 'Furniture',
        'Furniture', 'Sofas', '2', '500', '1000', '500', '500', '1000')''')
# Commit the transaction
conn.commit()

# Verify the updated record
cursor.execute('SELECT * FROM sales WHERE Product = "Sofas" LIMIT 10 OFFSET 5')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date      Day        Month      Year    Customer_Age  Age_Group     Customer_Gender    Country        State       Product_Category    Sub_Category    Product      Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
--------  ---------  -------  ------  --------------  ------------  -----------------  -------------  ----------  ------------------  --------------  ---------  ----------------  -----------  ------------  --------  ------  ---------
1/1/2020  Wednesday  January    2020              25  Young Adults  Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       700     500       1500
1/1/2020  Wednesday  January    2020              25  Young Adults  Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       700     500       1500
5/1/2020  Sunday     May        2020              45  Adults    

Bulk update in small batches

In [80]:
# Update Customers_Age add 1
cursor.execute('''UPDATE sales SET Customer_Age = Customer_Age + 1 WHERE Profit > 500''')
conn.commit()

# Verify the updated table
cursor.execute('''SELECT * FROM sales WHERE Profit BETWEEN 600 AND 900 LIMIT 10''')
rows = cursor.fetchall()
print(tabulate(rows, headers=column_names))

Date      Day        Month      Year    Customer_Age  Age_Group     Customer_Gender    Country        State       Product_Category    Sub_Category    Product      Order_Quantity    Unit_Cost    Unit_Price    Profit    Cost    Revenue
--------  ---------  -------  ------  --------------  ------------  -----------------  -------------  ----------  ------------------  --------------  ---------  ----------------  -----------  ------------  --------  ------  ---------
1/1/2020  Wednesday  January    2020              38  Young Adults  Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       700     500       1500
1/1/2020  Wednesday  January    2020              38  Young Adults  Female             United States  California  Furniture           Furniture       Sofas                     1          500          1000       700     500       1500
1/1/2020  Wednesday  January    2020              38  Young Adul

Aggregation in SQL
Aggregate functions (COUNT, SUM, AVG, MIN, MAX)
Grouping results using GROUP BY
Filtering grouped data with HAVING
Combining data from multiple tables using JOINs:
INNER JOIN
LEFT JOIN
RIGHT JOIN
FULL JOIN

COUNT (Total number)

In [91]:
# Count the number of records in the sales table
cursor.execute('SELECT COUNT(*) FROM sales')
count = cursor.fetchone()[0]
print(f"Total number of records in the sales table: {count}") # (count)

Total number of records in the sales table: 113055


Counting non-Null values

In [96]:
cursor.execute('SELECT COUNT(Product_Category) FROM sales')
prod_cat_count = cursor.fetchone()[0]
print(prod_cat_count)

113055


SUM (Total)

In [99]:
# Sum of Revenue
cursor.execute('SELECT SUM(Revenue) FROM sales')
revenue_sum = cursor.fetchone()[0]
print(f"Total Revenue: ${revenue_sum}") # (revenue_sum)

Total Revenue: $85290508.0


In [100]:
# Total Profit
cursor.execute('SELECT SUM(Profit) FROM sales')
profit_sum = cursor.fetchone()[0]
print(f"Total Profit: ${profit_sum}") # (profit_sum)

Total Profit: $32230700.0


AVERAGE (Mean)

In [101]:
# Average Revenue
cursor.execute('SELECT AVG(Revenue) FROM sales')
revenue_avg = cursor.fetchone()[0]
print(f"Average Revenue: ${revenue_avg}") # (revenue_avg)

Average Revenue: $754.4160629781965


In [106]:
# Average Customer_Age
cursor.execute('SELECT AVG(Customer_Age) FROM sales')
age_avg = cursor.fetchone()[0]
print(f"Average Customer_Age: {age_avg} years") # (age_avg)

Average Customer_Age: 45.920419264959534 years


Round (Approximation)

In [105]:
# Average Customer_Age
cursor.execute('SELECT ROUND(AVG(Customer_Age), 2) AS avg_age FROM sales')
age_avg = cursor.fetchone()[0]
print(f"Average Customer_Age: {age_avg} years") # (age_avg)

Average Customer_Age: 45.92 years


MIN (return minimum value)

In [107]:
# Minimum Revenue
cursor.execute('SELECT MIN(Revenue) FROM sales')
revenue_min = cursor.fetchone()[0]
print(f"Minimum Revenue: ${revenue_min}") # (revenue_min)

Minimum Revenue: $1500


MAX (maximum value)

In [109]:
# Oldest Customer_Age
cursor.execute('SELECT MAX(Customer_Age) FROM sales')
age_max = cursor.fetchone()[0]
print(f"Oldest Customer is {age_max} years") # (age_max)

Oldest Customer is 97 years


Aggregate with GROUP BY

In [110]:
# Sum of Revenue by Country
cursor.execute('SELECT Country, SUM(Revenue) AS Total_Revenue FROM sales GROUP BY Country')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Country", "Total_Revenue"]))

Country           Total_Revenue
--------------  ---------------
Australia           2.13021e+07
Canada              7.93574e+06
Country             0
France              8.43287e+06
Germany             8.9786e+06
United Kingdom      1.06462e+07
United States       2.7995e+07


In [122]:
# Sum of Revenue by Customer_Gender
cursor.execute('SELECT Customer_Gender, SUM(Revenue) AS Total_Revenue FROM sales GROUP BY Customer_Gender')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Customer_Gender", "Total_Revenue"]))


Customer_Gender      Total_Revenue
-----------------  ---------------
Customer_Gender        0
Female                 4.19491e+07
Male                   4.33414e+07


In [123]:
# Sum of Revenue by Product_Category
cursor.execute('SELECT Product_Category, SUM(Revenue) AS Total_Revenue FROM sales GROUP BY Product_Category')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Product_Category", "Total_Revenue"]))

Product_Category      Total_Revenue
------------------  ---------------
Accessories             1.5118e+07
Bikes                   6.17821e+07
Clothing                8.37088e+06
Furniture           19500
Product_Category        0


In [126]:
# Average Customer_Age by Product_Category
cursor.execute('SELECT Product_Category, ROUND(AVG(Customer_Age), 0) AS avg_age FROM sales GROUP BY Product_Category')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Product_Category", "Average Customer_Age"]))

Product_Category      Average Customer_Age
------------------  ----------------------
Accessories                             46
Bikes                                   45
Clothing                                46
Furniture                               56
Product_Category                        10


In [144]:
# Average Profit by Product_Category and Country
cursor.execute('''SELECT Product_Category, Country, ROUND(AVG(Profit), 2)   AS avg_profit
                  FROM sales GROUP BY Product_Category, Country LIMIT 10''')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Product_Category", "Country", "Average Profit"], tablefmt="fancy_grid"))

╒════════════════════╤════════════════╤══════════════════╕
│ Product_Category   │ Country        │   Average Profit │
╞════════════════════╪════════════════╪══════════════════╡
│ Accessories        │ Australia      │           112.48 │
├────────────────────┼────────────────┼──────────────────┤
│ Accessories        │ Canada         │           138.62 │
├────────────────────┼────────────────┼──────────────────┤
│ Accessories        │ France         │           116.51 │
├────────────────────┼────────────────┼──────────────────┤
│ Accessories        │ Germany        │           138.52 │
├────────────────────┼────────────────┼──────────────────┤
│ Accessories        │ United Kingdom │           141.08 │
├────────────────────┼────────────────┼──────────────────┤
│ Accessories        │ United States  │           123.6  │
├────────────────────┼────────────────┼──────────────────┤
│ Bikes              │ Australia      │           684.79 │
├────────────────────┼────────────────┼─────────────────

In [145]:
# Count the number of records in the sales table by Customer_Gender
cursor.execute('SELECT Customer_Gender, COUNT(*) FROM sales GROUP BY Customer_Gender')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Customer_Gender", "Count"], tablefmt="fancy_grid"))

╒═══════════════════╤═════════╕
│ Customer_Gender   │   Count │
╞═══════════════════╪═════════╡
│ Customer_Gender   │       1 │
├───────────────────┼─────────┤
│ Female            │   54736 │
├───────────────────┼─────────┤
│ Male              │   58318 │
╘═══════════════════╧═════════╛


In [146]:
# Total Profit by year and Country
cursor.execute('SELECT Year, SUM(Profit) AS Total_Profit FROM sales GROUP BY Year')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Year",  "Total Profit"], tablefmt="fancy_grid"))

╒════════╤════════════════╕
│ Year   │   Total Profit │
╞════════╪════════════════╡
│ 2011   │    2.8813e+06  │
├────────┼────────────────┤
│ 2012   │    2.95199e+06 │
├────────┼────────────────┤
│ 2013   │    5.95921e+06 │
├────────┼────────────────┤
│ 2014   │    5.86409e+06 │
├────────┼────────────────┤
│ 2015   │    7.52856e+06 │
├────────┼────────────────┤
│ 2016   │    7.03595e+06 │
├────────┼────────────────┤
│ 2020   │ 9600           │
├────────┼────────────────┤
│ Year   │    0           │
╘════════╧════════════════╛


In [131]:
# Maximum Revenue by Product_Category
cursor.execute('SELECT Product_Category, MAX(Revenue) AS Max_Revenue FROM sales GROUP BY Product_Category')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Product_Category", "Maximum Revenue"]))

Product_Category    Maximum Revenue
------------------  -----------------
Accessories         999
Bikes               9983
Clothing            998
Furniture           1000
Product_Category    Revenue


In [ ]:
# Maximum Profit by Product_Category
cursor.execute('SELECT Product_Category, MAX(Profit) AS Min_Profit FROM sales GROUP BY Product_Category')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Product_Category", "Maximum Profit"]))

Product_Category    Maximum Profit
------------------  ----------------
Accessories         99
Bikes               998
Clothing            996
Furniture           500
Product_Category    Profit


In [141]:
# Maximum Revenue by Date
cursor.execute('SELECT Date, MAX(Revenue) AS Max_Revenue FROM sales GROUP BY Year')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Date", "Maximum Revenue"], tablefmt="fancy_grid"))

╒════════════╤═══════════════════╕
│ Date       │ Maximum Revenue   │
╞════════════╪═══════════════════╡
│ 8/19/2011  │ 9983              │
├────────────┼───────────────────┤
│ 8/18/2012  │ 983               │
├────────────┼───────────────────┤
│ 12/26/2013 │ 999               │
├────────────┼───────────────────┤
│ 6/7/2014   │ 998               │
├────────────┼───────────────────┤
│ 8/26/2015  │ 999               │
├────────────┼───────────────────┤
│ 2/28/2016  │ 999               │
├────────────┼───────────────────┤
│ 5/1/2020   │ 1000              │
├────────────┼───────────────────┤
│ Date       │ Revenue           │
╘════════════╧═══════════════════╛


In [147]:
# Using HAVING to filter groups
cursor.execute('''SELECT Product_Category, Country, ROUND(AVG(Profit), 2)   AS avg_profit
                  FROM sales GROUP BY Product_Category, Country HAVING ROUND(AVG(Profit), 2) > 500''')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Product_Category", "Country", "Average Profit"], tablefmt="fancy_grid"))

╒════════════════════╤════════════════╤══════════════════╕
│ Product_Category   │ Country        │   Average Profit │
╞════════════════════╪════════════════╪══════════════════╡
│ Bikes              │ Australia      │           684.79 │
├────────────────────┼────────────────┼──────────────────┤
│ Bikes              │ Canada         │          1092.35 │
├────────────────────┼────────────────┼──────────────────┤
│ Bikes              │ France         │           689.8  │
├────────────────────┼────────────────┼──────────────────┤
│ Bikes              │ Germany        │           750.43 │
├────────────────────┼────────────────┼──────────────────┤
│ Bikes              │ United Kingdom │           843.97 │
├────────────────────┼────────────────┼──────────────────┤
│ Bikes              │ United States  │           849.37 │
├────────────────────┼────────────────┼──────────────────┤
│ Furniture          │ United States  │           533.33 │
╘════════════════════╧════════════════╧═════════════════

In [150]:
# Count the number of  state by Country
cursor.execute('SELECT Country, COUNT(DISTINCT State) FROM sales GROUP BY Country')
rows = cursor.fetchall()
print(tabulate(rows, headers=["Country", "Count"], tablefmt="fancy_grid"))

╒════════════════╤═════════╕
│ Country        │   Count │
╞════════════════╪═════════╡
│ Australia      │       5 │
├────────────────┼─────────┤
│ Canada         │       3 │
├────────────────┼─────────┤
│ Country        │       1 │
├────────────────┼─────────┤
│ France         │      16 │
├────────────────┼─────────┤
│ Germany        │       6 │
├────────────────┼─────────┤
│ United Kingdom │       1 │
├────────────────┼─────────┤
│ United States  │      22 │
╘════════════════╧═════════╛


Advanced SQL Concepts (Subqueries and Indexing)